# Inception Score

In [4]:
import tensorflow as tf
import numpy as np
from scipy.stats import entropy
from keras import models
from keras.applications.inception_v3 import InceptionV3, preprocess_input

In [7]:
def preprocess_images(images):
    # Convert to float32 and rescale to [0, 255] range if needed
    images = tf.image.resize(images, (299, 299))
    images = tf.image.grayscale_to_rgb(images)  # Convert grayscale to RGB
    images = preprocess_input(images)
    return images

def calculate_inception_score(images, n_split=10, eps=1E-16):
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
    images = preprocess_images(images)
    preds = model.predict(images)
    
    scores = []
    n_part = preds.shape[0] // n_split
    for i in range(n_split):
        part = preds[i * n_part:(i + 1) * n_part]
        p_y = np.mean(part, axis=0)
        
        # Ensure no zero probabilities
        p_y = np.clip(p_y, eps, 1 - eps)
        
        entropies = [entropy(p_y, np.clip(p, eps, 1 - eps)) for p in part]
        scores.append(np.exp(np.mean(entropies)))
    
    return np.mean(scores), np.std(scores)

In [12]:
generator = models.load_model("generator/generator-fashion-mnist-hingeloss.h5")

num_images = 1000
noise_dim = 100
noise = np.random.normal(0, 1, (num_images, noise_dim))
generated_images = generator.predict(noise)
generated_images = (generated_images + 1) / 2.0  

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


In [13]:
mean_is, std_is = calculate_inception_score(generated_images)
print(f"Inception Score: {mean_is} ± {std_is}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step
Inception Score: 1.0491445064544678 ± 0.002100131241604686
